In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_hotel       = pl.read_parquet(datadir / "hotel.parquet")
pl_reservation = pl.read_parquet(datadir / "reservation.parquet")

# 14章 ウィンドウ関数
## 14-1 順序のないウィンドウ関数
### Q: ホテルマスタに対してホテルのある市区町村の平均価格の付与


#### Awesome

In [ ]:
pl_hotel.with_columns(avg_price_within_city=
    # address_prefectureとaddress_cityごとにunit_priceの平均を算出
    pl.col("unit_price").mean().over(["address_prefecture", "address_city"])
)

### Q: 都道府県ごとのホテル数の構成比の算出


#### Awesome

In [ ]:
(
    pl_hotel
    #（1）都道府県ごとのホテル数を計算
    .group_by("address_prefecture").len()
    #（2）都道府県ごとのホテル数を、都道府県ごとのホテル数の合計（=全ホテル数）で除算して構成比を算出
    .with_columns(ratio=pl.col("len") / pl.col("len").sum())
)

## 14-2 順序のあるウィンドウ関数
### Q: 各顧客の一つ前と二つ前の予約の予約金額の取得


#### Awesome

In [ ]:
(
    pl_reservation
    #（1）reserved_atの昇順（時系列順）にソート
    .sort("reserved_at")
    .with_columns(
        #（2）customer_idごとに一つ前と二つ前の行のtotal_priceを取得
        total_price_at_prev=pl.col("total_price").shift(1).over("customer_id"),
        total_price_at_2prev=pl.col("total_price").shift(2).over("customer_id")
    )
)

### Q: 顧客ごとに予約順を付与


#### Awesome

In [ ]:
pl_reservation.with_columns(reservation_no=
    pl.col("reserved_at").rank(method="ordinal").over("customer_id"))

## 14-3 範囲のあるウィンドウ関数
### Q: 売上の移動平均の算出


#### Awesome

In [ ]:
(
    pl_reservation
    #（1）キャンセルを除去
    .filter(pl.col("status") != "canceled")
    #（2）checkout_dateを月単位に切り捨てた値ごとにtotal_priceの総和を計算
    .group_by(pl.col("checkout_date").dt.truncate("1mo"))
    .agg(pl.col("total_price").sum())
    #（3）checkout_dateの順にソート
    .sort("checkout_date")
    #（4）total_price（の月ごとの総和）の直近3行分の平均を計算
    .with_columns(moving_avarage_sales=pl.col("total_price").rolling_mean(3))
)

### Q: 顧客の過去の総予約金額を付与


#### Awesome

In [ ]:
(
    pl_reservation
    #（1）キャンセル済みのデータを除去
    .filter(pl.col("status") != "canceled")
    #（2）reserved_atの昇順（時系列順）にソート
    .sort("reserved_at")
    #（3）customer_idごとにtotal_priceを時系列順に1行分ずらし、累計和を計算
    .with_columns(past_total_spent=
        pl.col("total_price").shift(1).cum_sum().over("customer_id"))
)

### Q: 顧客の過去90日間の総予約金額を付与


#### Awesome

In [ ]:
(
    pl_reservation
    #（1）キャンセル済みのデータを除去
    .filter(pl.col("status") != "canceled")
    #（2）reserved_atの昇順（時系列順）にソート
    .sort("reserved_at")
    .with_columns(total_spent_last_90days=
        #（3）customer_idごとにreserved_atの日付順に90日前〜1日前のtotal_priceの総和を計算
        pl.col("total_price").rolling_sum("90d", by="reserved_at", closed='left').over("customer_id")
    )
)

## 14-4 指定列が最小／最大となる行の取得
### Q：顧客の初回予約時の情報の取得


#### Awesome

In [ ]:
(
    pl_reservation
    # （1）顧客ごとに予約順の番号を付与
    .with_columns(rn=pl.col("reserved_at").rank(method="ordinal").over("customer_id"))
    # （2）顧客の最初の予約のみ抽出
    .filter(pl.col("rn") == 1)
    .select(pl.col(["customer_id", "reserved_at", "hotel_id"]))
)

## 14-5 層別サンプリング
### Q: 都道府県別に層別サンプリング


#### Awesome

In [ ]:
(
    pl_hotel
    # address_prefecture列の値ごとに10%のデータをランダムサンプリング
    .group_by("address_prefecture").map_groups(lambda df: df.sample(fraction=0.1))
)

#### Awesome

In [ ]:
# address_prefecture列の値ごとに連番をランダムに振り
# その連番が全体の10％以下となる行のみ抽出
pl_hotel.filter(
    pl.col("hotel_id").rank(method="ordinal").shuffle().over("address_prefecture")
    <= pl.len().over("address_prefecture") * 0.1
)

## 14-6 グループ単位のランダムサンプリング
### Q: 顧客単位で予約履歴をランダムサンプリング


#### Awesome


In [ ]:
# （2）判定結果を用いて予約を抽出
pl_reservation.filter(
    # （1）顧客をサンプリングし、サンプリングした顧客の予約を判定
    pl.col("customer_id").is_in(pl.col("customer_id").unique().sample(fraction=0.01))
)